In [1]:
import pandas as pd
import numpy as np

from queries.getTestsStarts import ret as problemsStarts
from queries.getSubmissionsTime import ret as problemsEnds
from queries.getSubTimeForOpenAssess import ret as problemsAssessEnds
from queries.getVideoStartAndEnd import ret as videoStartsAndEnds
from queries.getSeqEvent import ret as seqEvents
from queries.getTextsInteractions import ret as textsInteractions
from queries.getHintsTime import ret as hintsTime 

from getThemeStartTimeForUser import getThemeStarts

In [2]:
problemsStartsDF=pd.DataFrame(problemsStarts())
problemsStartsDF=problemsStartsDF.loc[:,['course_id','problem_type','time','user_id']]
problemsStartsDF=problemsStartsDF.rename(index=str, columns={"problem_type": "action"})

In [3]:
problemsEndsDF=pd.DataFrame(problemsEnds())
problemsEndsDF=problemsEndsDF.loc[:,['course_id','time','user_id']]
problemsEndsDF['action'] = 'end ungraded'

In [4]:
problemsAssessEndsDF=pd.DataFrame(problemsAssessEnds())
problemsAssessEndsDF=problemsAssessEndsDF.loc[:,['course_id','time','user_id']]
problemsAssessEndsDF['action'] = 'end graded'

In [5]:
videoStartsAndEndsDF=pd.DataFrame(videoStartsAndEnds())
videoStartsAndEndsDF=videoStartsAndEndsDF.loc[:,['course_id','time','user_id']]
videoStartsAndEndsDF['action'] = 'video'

In [6]:
textsInteractionsDF=pd.DataFrame(textsInteractions())
textsInteractionsDF['course_id']=videoStartsAndEndsDF['course_id'][0]
textsInteractionsDF['action']='text'

In [7]:
getThemeStartsDF=pd.DataFrame(getThemeStarts())
getThemeStartsDF=getThemeStartsDF.loc[:,['course_id','time','user_id']]
getThemeStartsDF['action']='theme start'

In [8]:
hintsTimeDF=pd.DataFrame(hintsTime())
hintsTimeDF['action']='hint'

In [9]:
seqTimeDF=pd.DataFrame(seqEvents())

In [11]:
DF=pd.concat([problemsStartsDF, problemsEndsDF,problemsAssessEndsDF,videoStartsAndEndsDF,textsInteractionsDF,getThemeStartsDF,hintsTimeDF,seqTimeDF],ignore_index=True,sort=False)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [13]:
DF.to_csv(path_or_buf='users_sequence.csv',index=False)

In [14]:
#DF = pd.read_csv('users_sequence.csv', sep=',', encoding='utf-8')

In [17]:
users=np.unique(DF['user_id'])

In [ ]:
"""
textCounterInStudy=0
videoCounterInStudy=0
textCounterInProblems=0
videoCounterInProblems=0
hintCounterInProblems=0
gradedCounter=0
ungradedCounter=0
"""

In [ ]:
studyModelsDF=[]
problemModelsDF=[]

In [31]:
for user in users:
    userActions=DF.loc[DF['user_id']==user].sort_values(by='time')
    sequence=userActions['action'].values
    sequenceForModel=[]
    themeArr=[]
    
    gradedPosEnd=[]
    gradedPosStart=[]
    gradedSequences=[]
    
    ungradedPosEnd=[]
    ungradedPosStart=[]
    ungradedSequences=[]
    
    studySequence=[]
    
    for step in sequence:
        if step == 'theme start':
            if len(themeArr)>1:
                sequenceForModel.append(themeArr)
                themeArr=['theme start']
        elif len(themeArr)>=1:
            if step!=themeArr[-1] or step=='end graded' or step=='end ungraded':
                themeArr.append(step)
    sequenceForModel.append(themeArr)
    for actionsArr in sequenceForModel:
        if 'theme start' in actionsArr:
            actionsArr.remove('theme start')
            
        if not 'end graded' in actionsArr:
            while 'graded' in actionsArr:
                actionsArr.remove('graded')
        else:
            gradedPosEnd=[i for i, elem in enumerate(actionsArr) if elem == 'end graded']
            gradedPosStart=[i for i, elem in enumerate(actionsArr) if elem == 'graded']
            gradedPosStart.extend(gradedPosEnd)
            gradedPosStart.sort()
            gradedPosEnd.reverse()
            for end in gradedPosEnd:
                endPosInTmp=gradedPosStart.index(end)
                #end is position of graded end in sequence array, endPosInTmp - pos of graded end in indexArray
                if endPosInTmp>0:
                    startPos=gradedPosStart[endPosInTmp-1] #it is position of graded start in sequence array
                else:
                    startPos=end
                gradedSequences.append(actionsArr[startPos:(end+1)])     
                del actionsArr[startPos:end]                
                
        if not 'end ungraded' in actionsArr:
            while 'ungraded' in actionsArr:
                actionsArr.remove('ungraded')
        else:
            ungradedPosEnd=[i for i, elem in enumerate(actionsArr) if elem == 'end ungraded']
            ungradedPosStart=[i for i, elem in enumerate(actionsArr) if elem == 'ungraded']
            ungradedPosStart.extend(ungradedPosEnd)
            ungradedPosStart.sort()
            ungradedPosEnd.reverse()
            for end in ungradedPosEnd:
                endPosInTmp=gradedPosStart.index(end)
                #end is position of graded end in sequence array, endPosInTmp - pos of graded end in indexArray
                if endPosInTmp>0:
                    startPos=ungradedPosStart[endPosInTmp-1] #it is position of graded start in sequence array
                else:
                    startPos=end
                ungradedSequences.append(actionsArr[startPos:(end+1)])
                del actionsArr[startPos:end]
        while 'seq' in actionsArr:
            actionsArr.remove('seq')  
        while 'seq' in gradedSequences:
            gradedSequences.remove('seq')  
        while 'seq' in ungradedSequences:
            ungradedSequences.remove('seq')  
            
        gradedSequences=['CH' if x.find('end')!=-1 else x for x in gradedSequences]
        ungradedSequences=['CH' if x.find('end')!=-1 else x for x in ungradedSequences]
            
    while [] in sequenceForModel:
        sequenceForModel.remove([])

    studySeq=[]
    probSeq=[]
    for array in sequenceForModel:
        array.insert(0,'TS')
        array.append('TE')
        studySeq.extend(array)
    for array in gradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
    for array in ungradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
        
    studyStatesCounter=pd.DataFrame(columns=['TS','video','text','problem','TE'])
    studyStatesCounter['TS']=[studySeq.count('TS')]
    studyStatesCounter['TE']=[studySeq.count('TE')]
    studyStatesCounter['video']=[studySeq.count('video')]
    studyStatesCounter['text']=[studySeq.count('text')]
    studyStatesCounter['problem']=[studySeq.count('end graded')+studySeq.count('end ungraded')]
    
    study=pd.DataFrame(columns=['states','TS','video','text','problem','TE'])
    study['states']=['TS','video','text','problem','TE']
    study=study.set_index('states')
    study=study.fillna(0)
    study['TE']['TE']='*'
    
    oldState=False
    for state in studySeq:
        if state=='end graded' or state=='end ungraded':
            state='problem'
        if state!='TS' and oldState!='TE':
            study[state][oldState]+=1
        oldState=state    
    
    study=study.transpose()
    for col in study.columns:
        if studyStatesCounter[col][0]:
            study[col]/=studyStatesCounter[col][0]
    study=study.transpose()
    
    study['TS']='*'    
    study=study.transpose()
    study['TE']='*'
    study=study.transpose()
    study['TE']['TS']='*'    
    
    study=study.values
    studyModel=[]
    for array in study:
        studyModel.extend(array)
    while '*' in studyModel:
        studyModel.remove('*')    
        
    studyModelsDF.append(studyModel)
    
    problemStatesCounter=pd.DataFrame(columns=['PS','video','text','hint','PV','CH','PE'])
    problemStatesCounter['PS']=[probSeq.count('PS')]
    problemStatesCounter['PE']=[probSeq.count('PS')]
    problemStatesCounter['video']=[probSeq.count('video')]
    problemStatesCounter['text']=[probSeq.count('text')]
    problemStatesCounter['hint']=[probSeq.count('hint')]
    problemStatesCounter['CH']=[probSeq.count('CH')]
    problemStatesCounter['PV']=[probSeq.count('graded')+probSeq.count('ungraded')-PS_problem] ######осторожно

    problem=pd.DataFrame(columns=['states','PS','video','text','hint','prob view','CH','PE'])
    problem['states']=['PS','video','text','hint','prob view','CH','PE']
    problem=problem.set_index('states')
    problem=problem.fillna(0)
    
    problem['PS']='*'
    tmp=problem['PE']['CH']
    problem['PE']='*'
    problem['PE']['CH']=tmp
    
    problem=problem.transpose()
    problem['PE']='*'
    problem=problem.transpose()
    
    oldState=False
    for state in probSeq:
        if state=='graded' or state=='ungraded':
            state='prob view'
        if state!='PS' and oldState!='PE':
            problem[state][oldState]+=1
        oldState=state
        
    problem=problem.transpose()
    for col in problem.columns:
        if problemStatesCounter[col][0]:
            problem[col]/=problemStatesCounter[col][0]
    problem=problem.transpose()
    
    problem=problem.values
    problemModel=[]
    for array in problem:
        problemModel.extend(array)
    while '*' in studyModel:
        problemModel.remove('*')
    
    problemModelsDF.append(problemModel)
    
studyModelsDF=pd.DataFrame(studyModelsDF,columns=['TS_V','TS_T','TS_P',
                                                  'V_V','V_T','V_P','V_TE',
                                                  'T_V','T_T','T_P','T_TE',
                                                  'P_V','P_T','P_P','P_TE'])
problemModelsDF=pd.DataFrame(problemModelsDF,columns=['PS_V','PS_T','PS_H','PS_PV','PS_CH',
                                                      'V_V','V_T','V_H','V_PV','V_CH',
                                                      'T_V','T_T','T_H','T_PV','T_CH',
                                                      'H_V','H_T','H_H','H_PV','H_CH',
                                                      'PV_V','PV_T','PV_H','PV_PV','PV_CH',
                                                      'CH_V','CH_T','CH_H','CH_PV','CH_CH','CH_PE'     
                                                     ])    


IndentationError: unexpected indent (<ipython-input-31-201eda0502bd>, line 187)